In [5]:
from sklearn.datasets import load_wine, fetch_openml
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Load the Wine Dataset
wine = pd.read_csv('/Users/noshitha/Downloads/hw4/datasets/hw3_wine.csv')

In [8]:
df_wine = pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_wine.csv", delimiter="\t")

X_wine = df_wine.iloc[:, 1:]
y_wine = df_wine.iloc[:, 0]

In [11]:
df_wine = pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_wine.csv", delimiter="\t")

X_wine = df_wine.iloc[:, 1:]
y_wine = df_wine.iloc[:, 0]
# Convert categorical attributes to numerical inputs using one-hot encoding
X_wine = pd.get_dummies(df_wine.iloc[:, 1:])  # Exclude the target column
y_wine = df_wine.iloc[:, 0]
X_wine.head()

In [12]:
X_wine.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [13]:
y_wine.head()

0    1
1    1
2    1
3    1
4    1
Name: # class, dtype: int64

In [32]:
def normalize(X):
    if isinstance(X, pd.DataFrame):
        X = X.values
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_normalized = (X - mean) / std
    return X_normalized, mean, std

In [6]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, X):
        # Forward pass
        activations = X
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations, w) + b
            activations = self.sigmoid(z)
        return activations
    
    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            # Forward pass
            activations = X
            zs = []
            for w, b in zip(self.weights, self.biases):
                z = np.dot(activations, w) + b
                zs.append(z)
                activations = self.sigmoid(z)

            # Backward pass
            error = y - activations
            deltas = [error * self.sigmoid_derivative(activations)]
            for i in range(len(self.weights) - 1, 0, -1):
                delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(zs[i-1])
                deltas.append(delta)
            deltas.reverse()

            # Weight update
            for i in range(len(self.weights)):
                activations = np.atleast_2d(activations)
                self.weights[i] += learning_rate * np.dot(activations.T, deltas[i])
                self.biases[i] += learning_rate * deltas[i].sum(axis=0, keepdims=True)

# Example usage
nn = NeuralNetwork([2, 3, 1])  # Network with 2 input, 3 hidden, and 1 output neuron
X = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])  # Sample input data
y = np.array([[0.7], [0.9], [0.1]])  # Desired output
nn.train(X, y, learning_rate=0.1, epochs=1000)  # Train the network

# Make prediction on new data
prediction = nn.predict(np.array([[0.8, 0.9]]))
print(prediction)


[[0.56685714]]


In [7]:
X_train = np.array([0.13, 0.42])
y_train = np.array([0.9, 0.23])
nn.train(X_train, y_train, learning_rate=0.1, epochs=1000)  # Train the network

In [ ]:
prediction_x = nn.predict(np.array([[0.8, 0.9]]))
print(prediction_x)

## VALIDATION -  1

In [34]:
import numpy as np

# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Forward propagation
def forward_propagation(x, theta1, theta2):
    a1 = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)  # Add bias to input layer
    z2 = np.dot(a1, theta1.T)
    a2 = np.concatenate((np.ones((z2.shape[0], 1)), sigmoid(z2)), axis=1)  # Add bias to hidden layer
    z3 = np.dot(a2, theta2.T)
    a3 = sigmoid(z3)
    return a1, z2, a2, z3, a3

# Cost function
def compute_cost(y, a3):
    m = y.shape[0]
    J = -1/m * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3))
    return J

# Backpropagation
def backpropagation(x, y, theta1, theta2, a1, z2, a2, z3, a3, lam):
    m = y.shape[0]
    
    # Compute deltas
    delta3 = a3 - y
    delta2 = np.dot(delta3, theta2) * (a2 * (1 - a2))
    delta2 = delta2[:, 1:]  # Remove delta for bias neuron
    
    # Compute gradients
    grad1 = np.dot(delta2.T, a1)
    grad2 = np.dot(delta3.T, a2)
    
    # Regularization (excluding bias terms)
    grad1[:, 1:] += (lam / m) * theta1[:, 1:]
    grad2[:, 1:] += (lam / m) * theta2[:, 1:]
    
    # Average gradients
    grad1 /= m
    grad2 /= m
    
    return grad1, grad2

# Update weights
def update_weights(theta1, theta2, grad1, grad2, alpha):
    theta1 -= alpha * grad1
    theta2 -= alpha * grad2
    return theta1, theta2

# Function to print required outputs
def print_outputs(theta1, theta2, grad1, grad2, a3, y):
    print("Theta1:")
    print(theta1)
    print("\nTheta2:")
    print(theta2)
    print("\nRegularized Gradients of Theta1:")
    print(grad1)
    print("\nRegularized Gradients of Theta2:")
    print(grad2)
    print("\nFinal Prediction of Our Network:")
    print(a3)
    print("\nExpected Prediction:")
    print(y)

In [35]:
def example_1():
    # Initialize network architecture
    layers = [1, 2, 1]
    theta1 = np.array([[0.4, 0.1], [0.3, 0.2]])
    theta2 = np.array([[0.7, 0.5, 0.6]])

    # Training instances
    x = np.array([[0.13], [0.42]])
    y = np.array([[0.9], [0.23]])

    # Regularization parameter
    lam = 0.000

    # Hyperparameters
    alpha = 0.01  # Learning rate

    # Forward propagation
    a1, z2, a2, z3, a3 = forward_propagation(x, theta1, theta2)
    
    print("a1:",a1)
    print("z2:",z2)
    print("a2:",a2)
    print("z3:",z3)
    print("a3:",a3)
    
    # Calculate cost
    J = compute_cost(y, a3)
    
    print("Regularization cost: ",J)
    
    # Backpropagation
    grad1, grad2 = backpropagation(x, y, theta1, theta2, a1, z2, a2, z3, a3, lam)

    # Update weights
    theta1, theta2 = update_weights(theta1, theta2, grad1, grad2, alpha)

    # Print outputs
    print_outputs(theta1, theta2, grad1, grad2, a3, y)

In [36]:

def example_2():
    # Initialize network architecture
    layers = [2, 4, 3, 2]
    theta1 = np.array([[0.42, 0.15, 0.4],
                       [0.72, 0.1, 0.54],
                       [0.01, 0.19, 0.42],
                       [0.3, 0.35, 0.68]])
    theta2 = np.array([[0.21, 0.67, 0.14, 0.96, 0.87],
                       [0.87, 0.42, 0.2, 0.32, 0.89],
                       [0.03, 0.56, 0.8, 0.69, 0.09]])
    theta3 = np.array([[0.04, 0.87, 0.42, 0.53],
                       [0.17, 0.1, 0.95, 0.69]])

    # Training instances
    x = np.array([[0.32, 0.68], [0.83, 0.02]])
    y = np.array([[0.75, 0.98], [0.75, 0.28]])

    # Regularization parameter
    lam = 0.25

    # Hyperparameters
    alpha = 0.01  # Learning rate

    # Forward propagation
    a1, z2, a2, z3, a3, z4, a4 = forward_propagation(x, theta1, theta2, theta3)
    
    # Calculate cost
    J = compute_cost(y, a4)
    
    # Backpropagation
    grad1, grad2, grad3 = backpropagation(x, y, theta1, theta2, theta3, a1, z2, a2, z3, a3, z4, a4, lam)

    # Update weights
    theta1, theta2, theta3 = update_weights(theta1, theta2, theta3, grad1, grad2, grad3, alpha)

    # Print outputs
    print_outputs(theta1, theta2, theta3, grad1, grad2, grad3, a4, y)




In [37]:
print("EXAMPLE WITH GIVEN WEIGHTS :")
example_1()

EXAMPLE WITH GIVEN WEIGHTS :
a1: [[1.   0.13]
 [1.   0.42]]
z2: [[0.413 0.326]
 [0.442 0.384]]
a2: [[1.         0.601807   0.5807858 ]
 [1.         0.60873549 0.59483749]]
z3: [[1.34937498]
 [1.36127024]]
a3: [[0.79402743]
 [0.79596607]]
Regularization cost:  0.8209757904998143
Theta1:
[[0.39972649 0.09986671]
 [0.2996682  0.1998382 ]]

Theta2:
[[0.69770003 0.49859626 0.59862445]]

Regularized Gradients of Theta1:
[[0.02735127 0.01332866]
 [0.03317988 0.01618028]]

Regularized Gradients of Theta2:
[[0.22999675 0.1403743  0.13755523]]

Final Prediction of Our Network:
[[0.79402743]
 [0.79596607]]

Expected Prediction:
[[0.9 ]
 [0.23]]


In [38]:
def example_1_random():
    # Initialize network architecture
    layers = [1, 2, 1]
    # Randomly initialize weights
    theta1 = np.random.rand(layers[1], layers[0] + 1)  # +1 for bias
    theta2 = np.random.rand(layers[2], layers[1] + 1)  # +1 for bias

    # Training instances
    x = np.array([[0.13], [0.42]])
    y = np.array([[0.9], [0.23]])

    # Regularization parameter
    lam = 0.000

    # Hyperparameters
    alpha = 0.01  # Learning rate

    # Forward propagation
    a1, z2, a2, z3, a3 = forward_propagation(x, theta1, theta2)
    
    print("a1:", a1)
    print("z2:", z2)
    print("a2:", a2)
    print("z3:", z3)
    print("a3:", a3)
    
    # Calculate cost
    J = compute_cost(y, a3)
    
    print("Regularization cost: ", J)
    
    # Backpropagation
    grad1, grad2 = backpropagation(x, y, theta1, theta2, a1, z2, a2, z3, a3, lam)

    # Update weights
    theta1, theta2 = update_weights(theta1, theta2, grad1, grad2, alpha)

    # Print outputs
    print_outputs(theta1, theta2, grad1, grad2, a3, y)

print("EXAMPLE WITH RANDOM WEIGHTS :")
example_1_random()

EXAMPLE WITH RANDOM WEIGHTS :
a1: [[1.   0.13]
 [1.   0.42]]
z2: [[0.86333966 0.1051517 ]
 [1.04952884 0.15202402]]
a2: [[1.         0.70335793 0.52626373]
 [1.         0.74068441 0.53793298]]
z3: [[1.32993095]
 [1.35267414]]
a3: [[0.79082921]
 [0.79456647]]
Regularization cost:  0.819592029062117
Theta1:
[[0.77961582 0.64190253]
 [0.08413476 0.16162617]]

Theta2:
[[0.89627721 0.60472216 0.00796582]]

Regularized Gradients of Theta1:
[[0.02597301 0.01291158]
 [0.00052015 0.00025476]]

Regularized Gradients of Theta2:
[[0.22769784 0.17068972 0.12312315]]

Final Prediction of Our Network:
[[0.79082921]
 [0.79456647]]

Expected Prediction:
[[0.9 ]
 [0.23]]


In [ ]:
print("EXAMPLE WIHT GIVEN WEIGHTS : ")
example_2()

In [ ]:

def example_2_RANDOM():
    # Initialize network architecture
    layers = [2, 4, 3, 2]
    # Randomly initialize weights
    theta1 = np.random.rand(layers[1], layers[0] + 1)  # +1 for bias
    theta2 = np.random.rand(layers[2], layers[1] + 1)  # +1 for bias
    theta3 = np.random.rand(layers[3], layers[2] + 1)  # +1 for bias
    
#     theta1 = np.array([[0.42, 0.15, 0.4],
#                        [0.72, 0.1, 0.54],
#                        [0.01, 0.19, 0.42],
#                        [0.3, 0.35, 0.68]])
#     theta2 = np.array([[0.21, 0.67, 0.14, 0.96, 0.87],
#                        [0.87, 0.42, 0.2, 0.32, 0.89],
#                        [0.03, 0.56, 0.8, 0.69, 0.09]])
#     theta3 = np.array([[0.04, 0.87, 0.42, 0.53],
#                        [0.17, 0.1, 0.95, 0.69]])

    # Training instances
    x = np.array([[0.32, 0.68], [0.83, 0.02]])
    y = np.array([[0.75, 0.98], [0.75, 0.28]])

    # Regularization parameter
    lam = 0.25

    # Hyperparameters
    alpha = 0.01  # Learning rate

    # Forward propagation
    a1, z2, a2, z3, a3, z4, a4 = forward_propagation(x, theta1, theta2, theta3)
    
    # Calculate cost
    J = compute_cost(y, a4)
    
    # Backpropagation
    grad1, grad2, grad3 = backpropagation(x, y, theta1, theta2, theta3, a1, z2, a2, z3, a3, z4, a4, lam)

    # Update weights
    theta1, theta2, theta3 = update_weights(theta1, theta2, theta3, grad1, grad2, grad3, alpha)

    # Print outputs
    print_outputs(theta1, theta2, theta3, grad1, grad2, grad3, a4, y)




In [40]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, theta1=None, theta2=None):
        self.layers = layers
        if theta1 is None:
            self.theta1 = np.array([[0.4, 0.1], [0.3, 0.2]])  # Initial Theta1
        else:
            self.theta1 = theta1
        if theta2 is None:
            self.theta2 = np.array([[0.7, 0.5, 0.6]])  # Initial Theta2
        else:
            self.theta2 = theta2

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_propagation(self, x):
        a1 = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)  # Add bias to input layer
        z2 = np.dot(a1, self.theta1.T)
        a2 = np.concatenate((np.ones((z2.shape[0], 1)), self.sigmoid(z2)), axis=1)  # Add bias to hidden layer
        z3 = np.dot(a2, self.theta2.T)
        a3 = self.sigmoid(z3)
        return a1, z2, a2, z3, a3

    def compute_cost(self, y, a3):
        m = y.shape[0]
        J = -1/m * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3))
        return J

    def train(self, X, Y):
        for i, (x, y) in enumerate(zip(X, Y), 1):
            print(f"Training instance {i}")
            print(f"    x: {x}")
            print(f"    y: {y}")

            # Forward propagation
            a1, z2, a2, z3, a3 = self.forward_propagation(x.reshape(1, -1))

            # Print forward propagation results
            print("\nForward propagation:")
            print(f"    a1: {a1}")
            print(f"    z2: {z2}")
            print(f"    a2: {a2}")
            print(f"    z3: {z3}")
            print(f"    a3: {a3}")

            # Compute cost
            J = self.compute_cost(y.reshape(1, -1), a3)
            print(f"\nPredicted output: {a3}")
            print(f"Expected output: {y}")
            print(f"Cost, J: {J}\n")

# Example usage
nn_example1 = NeuralNetwork([1, 2, 1])  # Example 1 network with 1 input, 2 hidden, and 1 output neuron
nn_example2 = NeuralNetwork([2, 4, 3, 2])  # Example 2 network with 2 input, 4 hidden, 3 hidden, and 2 output neurons

X_example1 = np.array([[0.13], [0.42]])  # Example 1 input data
Y_example1 = np.array([[0.9], [0.23]])  # Example 1 desired output

print("Example 1:")
nn_example1.train(X_example1, Y_example1)

Example 1:
Training instance 1
    x: [0.13]
    y: [0.9]

Forward propagation:
    a1: [[1.   0.13]]
    z2: [[0.413 0.326]]
    a2: [[1.        0.601807  0.5807858]]
    z3: [[1.34937498]]
    a3: [[0.79402743]]

Predicted output: [[0.79402743]]
Expected output: [0.9]
Cost, J: 0.36557477431084995

Training instance 2
    x: [0.42]
    y: [0.23]

Forward propagation:
    a1: [[1.   0.42]]
    z2: [[0.442 0.384]]
    a2: [[1.         0.60873549 0.59483749]]
    z3: [[1.36127024]]
    a3: [[0.79596607]]

Predicted output: [[0.79596607]]
Expected output: [0.23]
Cost, J: 1.2763768066887786



In [47]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, X):
        # Forward pass
        activations = X
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations, w) + b
            activations = self.sigmoid(z)
        return activations
    
    def train(self, X, Y, lam):
        total_cost = 0.0
        total_reg_cost = 0.0  # Variable to store the total regularization cost
        m = len(X)
        for i, (x, y) in enumerate(zip(X, Y), 1):
            print(f"Training instance {i}")
            print(f"    x: {x}")
            print(f"    y: {y}")

            # Forward propagation
            activations = x
            zs = []
            for w, b in zip(self.weights, self.biases):
                z = np.dot(activations, w) + b
                zs.append(z)
                activations = self.sigmoid(z)

            # Compute cost
            prediction = activations
            cost = -y * np.log(prediction) - (1 - y) * np.log(1 - prediction)
            instance_cost = np.sum(cost)

            # Regularization term
            reg_term = sum(np.sum(w[:, 1:] ** 2) for w in self.weights)
            total_reg_cost += reg_term  # Accumulate regularization cost for this instance

            # Update total cost with regularization
            total_cost += instance_cost

            # Print forward propagation results
            print("\nForward propagation:")
            print(f"    a1: {x}")
            for i, (z, a) in enumerate(zip(zs, activations), 2):
                print(f"    z{i}: {z}")
                print(f"    a{i}: {a}")

            # Predicted output
            prediction = activations
            print(f"\nPredicted output: {prediction}")
            print(f"Expected output: {y}")

            # Print cost associated with instance
            print(f"Cost, J, associated with instance {i}: {instance_cost}")

        # Average cost over all training instances
        avg_cost = total_cost / m

        # Calculate final regularization cost
        final_reg_cost = (lam / (2 * m)) * total_reg_cost

        print("FINAL REG COST :", final_reg_cost)


# Example usage
nn_example1 = NeuralNetwork([1, 2, 1])  # Example 1 network with 1 input, 2 hidden, and 1 output neuron
nn_example2 = NeuralNetwork([2, 4, 3, 2])  # Example 2 network with 2 input, 4 hidden, 3 hidden, and 2 output neurons

X_example1 = np.array([[0.13], [0.42]])  # Example 1 input data
Y_example1 = np.array([[0.9], [0.23]])  # Example 1 desired output

X_example2 = np.array([[0.32, 0.68], [0.83, 0.02]])  # Example 2 input data
Y_example2 = np.array([[0.75, 0.98], [0.75, 0.28]])  # Example 2 desired output

print("Example 1:")
nn_example1.train(X_example1, Y_example1,lam=0.00)

print("\nExample 2:")
nn_example2.train(X_example2, Y_example2,lam=0.25)

Example 1:
Training instance 1
    x: [0.13]
    y: [0.9]

Forward propagation:
    a1: [0.13]
    z2: [[0.00344647 0.07615083]]
    a2: [0.68343712]

Predicted output: [[0.68343712]]
Expected output: [0.9]
Cost, J, associated with instance 2: 0.45758190338572863
Training instance 2
    x: [0.42]
    y: [0.23]

Forward propagation:
    a1: [0.42]
    z2: [[0.01113475 0.24602574]]
    a2: [0.68905122]

Predicted output: [[0.68905122]]
Expected output: [0.23]
Cost, J, associated with instance 2: 0.985118967292997
FINAL REG COST : 0.0

Example 2:
Training instance 1
    x: [0.32 0.68]
    y: [0.75 0.98]

Forward propagation:
    a1: [0.32 0.68]
    z2: [[0.76441045 0.46840202 0.80025051 0.79023581]]
    a2: [0.74723525 0.83655055]

Predicted output: [[0.74723525 0.83655055]]
Expected output: [0.75 0.98]
Cost, J, associated with instance 2: 0.7734794243084668
Training instance 2
    x: [0.83 0.02]
    y: [0.75 0.28]

Forward propagation:
    a1: [0.83 0.02]
    z2: [[0.34101033 0.54165127 

In [6]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, X):
        # Forward pass
        activations = X
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations, w) + b
            activations = self.sigmoid(z)
        return activations
    
    def train(self, X, Y, lam):
        total_cost = 0.0
        total_reg_cost = 0.0  # Variable to store the total regularization cost
        m = len(X)
        for i, (x, y) in enumerate(zip(X, Y), 1):
            print(f"Training instance {i}")
            print(f"    x: {x}")
            print(f"    y: {y}")

            # Forward propagation
            activations = x
            zs = []
            for w, b in zip(self.weights, self.biases):
                z = np.dot(activations, w) + b
                zs.append(z)
                activations = self.sigmoid(z)

            # Compute cost
            prediction = activations
            cost = -y * np.log(prediction) - (1 - y) * np.log(1 - prediction)
            instance_cost = np.sum(cost)

            # Regularization term
            reg_term = sum(np.sum(w[:, 1:] ** 2) for w in self.weights)
            total_reg_cost += reg_term  # Accumulate regularization cost for this instance

            # Update total cost with regularization
            total_cost += instance_cost

            # Print forward propagation results
            print("\nForward propagation:")
            print(f"    a1: {x}")
            for i, (z, a) in enumerate(zip(zs, activations), 2):
                print(f"    z{i}: {z}")
                print(f"    a{i}: {a}")

            # Predicted output
            prediction = activations
            print(f"\nPredicted output: {prediction}")
            print(f"Expected output: {y}")

            # Print cost associated with instance
            print(f"Cost associated with instance {i}: {instance_cost}")

        # Average cost over all training instances
        avg_cost = total_cost / m

        # Calculate final regularization cost
        final_reg_cost = (lam / (2 * m)) * total_reg_cost

        # Add regularization term to the total cost
        total_cost += final_reg_cost



# Example usage
nn_example1 = NeuralNetwork([1, 2, 1]) 
nn_example2 = NeuralNetwork([2, 4, 3, 2])  
X_example1 = np.array([[0.13], [0.42]])  # Example 1 input data
Y_example1 = np.array([[0.9], [0.23]])  # Example 1 desired output

X_example2 = np.array([[0.32, 0.68], [0.83, 0.02]])  # Example 2 input data
Y_example2 = np.array([[0.75, 0.98], [0.75, 0.28]])  # Example 2 desired output

print("Example 1:")
nn_example1.train(X_example1, Y_example1, lam=0.25)  # Using regularization parameter lambda=0.25

print("\nExample 2:")
nn_example2.train(X_example2, Y_example2, lam=0.25)  # Using regularization parameter lambda=0.25


Example 1:
Training instance 1
    x: [0.13]
    y: [0.9]

Forward propagation:
    a1: [0.13]
    z2: [[0.0478408  0.02195366]]
    a2: [0.59203681]

Predicted output: [[0.59203681]]
Expected output: [0.9]
Cost associated with instance 2: 0.561425654399335
Training instance 2
    x: [0.42]
    y: [0.23]

Forward propagation:
    a1: [0.42]
    z2: [[0.15456259 0.07092722]]
    a2: [0.59635407]

Predicted output: [[0.59635407]]
Expected output: [0.23]
Cost associated with instance 2: 0.8174490057991711

Example 2:
Training instance 1
    x: [0.32 0.68]
    y: [0.75 0.98]

Forward propagation:
    a1: [0.32 0.68]
    z2: [[0.92849484 0.62270586 0.41044249 0.62668405]]
    a2: [0.82233176 0.87781949]

Predicted output: [[0.82233176 0.87781949]]
Expected output: [0.75 0.98]
Cost associated with instance 2: 0.7484209743143136
Training instance 2
    x: [0.83 0.02]
    y: [0.75 0.28]

Forward propagation:
    a1: [0.83 0.02]
    z2: [[0.69463309 0.42657113 0.42826308 0.45767375]]
    a2: [0

In [53]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, X):
        # Forward pass
        activations = X
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations, w) + b
            activations = self.sigmoid(z)
        return activations
    
    def train(self, X, Y, lam, epsilon, max_iterations):
        total_cost = 0.0
        total_reg_cost = 0.0  # Variable to store the total regularization cost
        m = len(X)
        prev_cost = float('inf')
        iteration = 0
        
        while True:
            iteration += 1
            print(f"Iteration {iteration}")
            for i, (x, y) in enumerate(zip(X, Y), 1):
                # Forward propagation
                activations = x
                zs = []
                for w, b in zip(self.weights, self.biases):
                    z = np.dot(activations, w) + b
                    zs.append(z)
                    activations = self.sigmoid(z)

                # Compute cost
                prediction = activations
                cost = -y * np.log(prediction) - (1 - y) * np.log(1 - prediction)
                instance_cost = np.sum(cost)

                # Regularization term
                reg_term = sum(np.sum(w[:, 1:] ** 2) for w in self.weights)
                total_reg_cost += reg_term  # Accumulate regularization cost for this instance

                # Update total cost with regularization
                total_cost += instance_cost
            
            # Average cost over all training instances
            avg_cost = total_cost / m

            # Calculate final regularization cost
            final_reg_cost = (lam / (2 * m)) * total_reg_cost

            # Print total cost
            print(f"Total cost: {avg_cost + final_reg_cost}")

            # Check for improvement in cost function
            if abs(prev_cost - (avg_cost + final_reg_cost)) < epsilon:
                print("Stopping criteria reached: Improvement in cost function less than epsilon.")
                break
            
            # Check for maximum number of iterations
            if iteration >= max_iterations:
                print("Stopping criteria reached: Maximum number of iterations reached.")
                break
            
            prev_cost = avg_cost + final_reg_cost
            
            # Reset total cost and regularization cost for next iteration
            total_cost = 0.0
            total_reg_cost = 0.0
            
            # Shuffle training data for next iteration
            combined = list(zip(X, Y))
            np.random.shuffle(combined)
            X, Y = zip(*combined)


# Example usage
nn_example1 = NeuralNetwork([1, 2, 1])  # Example 1 network with 1 input, 2 hidden, and 1 output neuron
nn_example2 = NeuralNetwork([2, 4, 3, 2])  # Example 2 network with 2 input, 4 hidden, 3 hidden, and 2 output neurons

X_example1 = np.array([[0.13], [0.42]])  # Example 1 input data
Y_example1 = np.array([[0.9], [0.23]])  # Example 1 desired output

X_example2 = np.array([[0.32, 0.68], [0.83, 0.02]])  # Example 2 input data
Y_example2 = np.array([[0.75, 0.98], [0.75, 0.28]])  # Example 2 desired output

print("Example 1:")
nn_example1.train(X_example1, Y_example1, lam=0.00, epsilon=0.0001, max_iterations=100)

print("\nExample 2:")
nn_example2.train(X_example2, Y_example2, lam=0.25, epsilon=0.0001, max_iterations=100)


Example 1:
Iteration 1
Total cost: 0.7199456326315257
Iteration 2
Total cost: 0.7199456326315257
Stopping criteria reached: Improvement in cost function less than epsilon.

Example 2:
Iteration 1
Total cost: 2.0859261907417728
Iteration 2
Total cost: 2.0859261907417728
Stopping criteria reached: Improvement in cost function less than epsilon.


y_encoded:       1  2  3
0    1  0  0
1    1  0  0
2    1  0  0
3    1  0  0
4    1  0  0
..  .. .. ..
173  0  0  1
174  0  0  1
175  0  0  1
176  0  0  1
177  0  0  1

[178 rows x 3 columns]
Iteration 1
Total cost: -0.04882722492668656
Iteration 2
Total cost: -0.04882722492668656
Stopping criteria reached: Improvement in cost function less than epsilon.
Accuracy: 0.08333333333333333
F1 Score: 1.0


In [105]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Define function for normalization
def normalize(X):
    if isinstance(X, pd.DataFrame):
        X = X.values
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_normalized = (X - mean) / std
    return X_normalized, mean, std

# Define function to calculate accuracy
def accuracy_score(y_true, y_pred):
    correct = sum(1 for yt, yp in zip(y_true, y_pred) if yt == yp)
    return correct / len(y_true)

# Define function to calculate F1 score
def f1_score(y_true, y_pred):
    tp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 1)
    fp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 0 and yp == 1)
    fn = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 0)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    return 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

# Define function to evaluate model performance
def evaluate_model_performance(model, X_train, y_train, X_test, y_test):
    model.train(X_train, y_train, learning_rate=0.01, lam=0.01)  # Example: Training the model with lambda=0.01
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, f1

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.rand(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, X):
        # Forward pass
        activations = X
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations, w) + b
            activations = self.sigmoid(z)
        return activations
    
    def train(self, X, Y, learning_rate, lam):
        total_cost = 0.0
        total_reg_cost = 0.0
        m = len(X)
        for i, (x, y) in enumerate(zip(X, Y), 1):
            activations = x
            zs = []
            activation_layers = [activations]  # Store activations for all layers
            # Forward pass
            for w, b in zip(self.weights, self.biases):
                z = np.dot(activations, w) + b
                zs.append(z)
                activations = self.sigmoid(z)
                activation_layers.append(activations)

            prediction = activations
            cost = -y * np.log(prediction) - (1 - y) * np.log(1 - prediction)
            instance_cost = np.sum(cost)

            reg_term = sum(np.sum(w[:, 1:] ** 2) for w in self.weights)
            total_reg_cost += reg_term

            total_cost += instance_cost

            delta = prediction - y
            for i in range(len(self.weights)-1, -1, -1):
                delta = np.dot(delta, self.weights[i].T) * self.sigmoid_derivative(activation_layers[i+1])
                self.weights[i] -= (learning_rate * np.dot(activation_layers[i].T, delta) + (lam / m) * self.weights[i])
                self.biases[i] -= learning_rate * np.sum(delta, axis=0)

        avg_cost = total_cost / m
        final_reg_cost = (lam / (2 * m)) * total_reg_cost
        print("FINAL REG COST :", final_reg_cost)


# Load dataset
df_wine = pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_wine.csv", delimiter="\t")

# Preprocess data
X_wine = pd.get_dummies(df_wine.iloc[:, 1:])  # Exclude the target column
y_wine = df_wine.iloc[:, 0]

# Normalize data
X_wine_normalized, mean, std = normalize(X_wine)

# Reshape y_wine to a 2D array
y_wine = y_wine.values.reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the target variable
y_encoded = encoder.fit_transform(y_wine)

# Convert y_encoded to a DataFrame
y_encoded_df = pd.DataFrame(y_encoded, columns=['y1', 'y2', 'y3'])

# Define model architecture
architectures = [
    [X_wine_normalized.shape[1], 5, 3, 3],  # Example architecture
]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_wine_normalized, y_encoded, test_size=0.2, random_state=42)

# Instantiate the NeuralNetwork class with the desired architecture
model = NeuralNetwork(architectures[0])

# Train the model
model.train(X_train, y_train, learning_rate=0.01, lam=0.01)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation results
print("Accuracy:", accuracy)
print("F1 Score:", f1)


ValueError: operands could not be broadcast together with shapes (1,5) (1,3) 

In [93]:
# Load dataset
df_wine = pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_wine.csv", delimiter="\t")

# Preprocess data
X_wine = pd.get_dummies(df_wine.iloc[:, 1:])  # Exclude the target column
y_wine = df_wine.iloc[:, 0]

# Normalize data
X_wine_normalized, mean, std = normalize(X_wine)

In [94]:
y_wine.unique()

array([1, 2, 3])

In [95]:
df_wine.head()

,# class,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [96]:
y_wine.head()

0    1
1    1
2    1
3    1
4    1
Name: # class, dtype: int64

In [4]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_pass(self, X):
        activations = [X]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations[-1], w) + b
            activations.append(self.sigmoid(z))
        return activations
    
    def backward_pass(self, X, Y, activations):
        deltas = [(activations[-1] - Y) * self.sigmoid_derivative(activations[-1])]
        for i in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(activations[i])
            deltas.insert(0, delta)
        return deltas
    
    def compute_gradients(self, activations, deltas):
        gradients_weights = [np.dot(activations[i].T, deltas[i]) for i in range(len(self.layers) - 1)]
        gradients_biases = [np.sum(deltas[i], axis=0) for i in range(len(self.layers) - 1)]
        return gradients_weights, gradients_biases
    
    def update_weights(self, gradients_weights, gradients_biases, learning_rate):
        self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_weights)]
        self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_biases)]
    
    def train(self, X, Y, learning_rate, lam, max_iterations, epsilon):
        for iteration in range(max_iterations):
            activations = self.forward_pass(X)
            deltas = self.backward_pass(X, Y, activations)
            gradients_weights, gradients_biases = self.compute_gradients(activations, deltas)
            self.update_weights(gradients_weights, gradients_biases, learning_rate)
            # Compute cost function
            J = np.mean(np.square(activations[-1] - Y))
            print(f"Iteration {iteration+1}, Cost: {J}")
            # Check for convergence
            if J < epsilon:
                print("Converged!")
                break

# Example 1
print("Example 1:")
print("Regularization parameter lambda=0.000")

# Define the structure of the neural network
nn_example_1 = NeuralNetwork([1, 2, 1])

# Initialize Theta1 and Theta2 with provided values
nn_example_1.weights[0] = np.array([[0.4, 0.1], [0.3, 0.2]])
nn_example_1.weights[1] = np.array([[0.7, 0.5]])

# Define the training data
X_train_example_1 = np.array([[0.13, 0.9]])
Y_train_example_1 = np.array([[0.5]])  # Placeholder target output

# Train the neural network
nn_example_1.train(X_train_example_1, Y_train_example_1, learning_rate=0.1, lam=0.0, max_iterations=1000, epsilon=0.005)

Example 1:
Regularization parameter lambda=0.000


ValueError: shapes (1,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

TypeError: 'tuple' object is not callable

In [5]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_pass(self, X):
        activations = [X]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations[-1], w) + b
            activations.append(self.sigmoid(z))
        return activations
    
    def backward_pass(self, X, Y, activations):
        deltas = [(activations[-1] - Y) * self.sigmoid_derivative(activations[-1])]
        for i in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(activations[i])
            deltas.insert(0, delta)
        return deltas
    
    def compute_gradients(self, activations, deltas):
        gradients_weights = [np.dot(activations[i].T, deltas[i]) for i in range(len(self.layers) - 1)]
        gradients_biases = [np.sum(deltas[i], axis=0) for i in range(len(self.layers) - 1)]
        print("gradient_weights: ",gradients_weights)
        print("gradients_biases: ",gradients_biases)
        return gradients_weights, gradients_biases
    
    def update_weights(self, gradients_weights, gradients_biases, learning_rate):
        self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_weights)]
        self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_biases)]
    
    def train(self, X, Y, learning_rate, lam, max_iterations, epsilon):
        for iteration in range(max_iterations):
            activations = self.forward_pass(X)
            deltas = self.backward_pass(X, Y, activations)
            gradients_weights, gradients_biases = self.compute_gradients(activations, deltas)
            self.update_weights(gradients_weights, gradients_biases, learning_rate)
            # Compute cost function
            J = np.mean(np.square(activations[-1] - Y))
            print(f"Iteration {iteration+1}, Cost: {J}")
            # Check for convergence
            if J < epsilon:
                print("Converged!")
                break

    def accuracy(self, y_true, y_pred):
        correct = np.sum(y_true == y_pred)
        return correct / len(y_true)

    def f1_score(self, y_true, y_pred):
        tp = np.sum(y_true & y_pred)
        fp = np.sum((~y_true) & y_pred)
        fn = np.sum(y_true & (~y_pred))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        return f1

    def evaluate(self, X_test, y_test):
        activations = self.forward_pass(X_test)[-1]
        y_pred = (activations > 0.5).astype(int)
        acc = self.accuracy(y_test, y_pred)
        f1 = self.f1_score(y_test, y_pred)
        return acc, f1

# Example usage
# Create a neural network with 2 input neurons, 3 hidden neurons, and 1 output neuron
nn = NeuralNetwork([2, 3, 1])

# Generate some dummy data for training
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y_train = np.array([[0], [1], [1], [0]])

# Train the neural network
nn.train(X_train, Y_train, learning_rate=0.1, lam=0.0, max_iterations=1000, epsilon=0.005)

# Generate some dummy data for testing
X_test = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y_test = np.array([[0], [1], [1], [0]])

# Evaluate the trained model
accuracy, f1_score = nn.evaluate(X_test, Y_test)
print("Accuracy:", accuracy)
print("F1 Score:", f1_score)


gradient_weights:  [array([[ 0.00476952,  0.00076406, -0.00162829],
       [ 0.01648216,  0.00303619, -0.00192312]]), array([[-0.04202719],
       [-0.05293396],
       [-0.04772457]])]
gradients_biases:  [array([ 0.01874646,  0.00397823, -0.0041006 ]), array([-0.09949241])]
Iteration 1, Cost: 0.2620492738553714
gradient_weights:  [array([[ 0.00440974,  0.00064801, -0.00159181],
       [ 0.01615018,  0.00288054, -0.00190987]]), array([[-0.04064788],
       [-0.05075277],
       [-0.04602825]])]
gradients_biases:  [array([ 0.01794559,  0.00369349, -0.0040653 ]), array([-0.09587024])]
Iteration 2, Cost: 0.26119324302246755
gradient_weights:  [array([[ 0.00406447,  0.00053957, -0.00155146],
       [ 0.01582726,  0.00273338, -0.00189252]]), array([[-0.03930246],
       [-0.04862567],
       [-0.04436967]])]
gradients_biases:  [array([ 0.01717104,  0.00342531, -0.00402111]), array([-0.09233135])]
Iteration 3, Cost: 0.2603989674328
gradient_weights:  [array([[ 0.00373349,  0.00043834, -0.001

gradients_biases:  [array([ 1.08382682e-03, -1.57479911e-03, -5.57827003e-05]), array([-0.00082748])]
Iteration 286, Cost: 0.2496730446689419
gradient_weights:  [array([[-3.86372914e-04, -1.67208622e-03,  5.51890411e-04],
       [ 8.09279154e-03,  4.86586465e-04,  5.11314418e-05]]), array([[-0.00175858],
       [ 0.00426487],
       [-0.00081084]])]
gradients_biases:  [array([ 1.08956038e-03, -1.57772281e-03, -5.58789201e-05]), array([-0.00082863])]
Iteration 287, Cost: 0.24966828050504766
gradient_weights:  [array([[-3.76687654e-04, -1.67461036e-03,  5.50965642e-04],
       [ 8.09602326e-03,  4.87686732e-04,  5.18771488e-05]]), array([[-0.00174676],
       [ 0.0042615 ],
       [-0.00081071]])]
gradients_biases:  [array([ 1.09528790e-03, -1.58064459e-03, -5.59755817e-05]), array([-0.00082978])]
Iteration 288, Cost: 0.2496635160382525
gradient_weights:  [array([[-3.67007445e-04, -1.67713267e-03,  5.50042189e-04],
       [ 8.09927283e-03,  4.88789002e-04,  5.26228472e-05]]), array([[-0.

       [-0.00084622]])]
gradients_biases:  [array([ 2.18881565e-03, -2.17151372e-03, -8.66842924e-05]), array([-0.00115721])]
Iteration 503, Cost: 0.24855203404344195
gradient_weights:  [array([[ 0.00168074, -0.00219984,  0.00038121],
       [ 0.00918709,  0.00077878,  0.00021878]]), array([[ 0.00088813],
       [ 0.00372907],
       [-0.00084668]])]
gradients_biases:  [array([ 2.19322243e-03, -2.17411870e-03, -8.68758055e-05]), array([-0.00115909])]
Iteration 504, Cost: 0.24854608386063787
gradient_weights:  [array([[ 0.00169046, -0.00220228,  0.00038056],
       [ 0.00919387,  0.0007804 ,  0.00021961]]), array([[ 0.00090122],
       [ 0.00372744],
       [-0.00084714]])]
gradients_biases:  [array([ 2.19762199e-03, -2.17672261e-03, -8.70678050e-05]), array([-0.00116098])]
Iteration 505, Cost: 0.24854012283900329
gradient_weights:  [array([[ 0.00170019, -0.00220471,  0.00037991],
       [ 0.00920066,  0.00078202,  0.00022044]]), array([[ 0.00091433],
       [ 0.00372581],
       [-0.00

gradients_biases:  [array([ 0.00301253, -0.00287615, -0.00016307]), array([-0.00182115])]
Iteration 786, Cost: 0.246210745591753
gradient_weights:  [array([[ 0.0047434 , -0.00296105,  0.0002484 ],
       [ 0.01165098,  0.00133406,  0.00050568]]), array([[ 0.00523376],
       [ 0.00353129],
       [-0.00109472]])]
gradients_biases:  [array([ 0.00301324, -0.00287855, -0.00016343]), array([-0.00182397])]
Iteration 787, Cost: 0.2461992236299217
gradient_weights:  [array([[ 0.00475563, -0.00296412,  0.00024812],
       [ 0.01166129,  0.00133635,  0.00050696]]), array([[ 0.0052517 ],
       [ 0.0035315 ],
       [-0.00109604]])]
gradients_biases:  [array([ 0.00301393, -0.00288095, -0.0001638 ]), array([-0.0018268])]
Iteration 788, Cost: 0.24618767159450367
gradient_weights:  [array([[ 0.00476787, -0.0029672 ,  0.00024785],
       [ 0.0116716 ,  0.00133864,  0.00050824]]), array([[ 0.00526967],
       [ 0.00353171],
       [-0.00109737]])]
gradients_biases:  [array([ 0.0030146 , -0.00288335, 

In [27]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_pass(self, X):
        activations = [X]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations[-1], w) + b
            activations.append(self.sigmoid(z))
        return activations
    
    def backward_pass(self, X, Y, activations):
        deltas = [(activations[-1] - Y) * self.sigmoid_derivative(activations[-1])]
        for i in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(activations[i])
            deltas.insert(0, delta)
        return deltas
    
    def compute_gradients(self, activations, deltas):
        gradients_weights = [np.dot(activations[i].T, deltas[i]) for i in range(len(self.layers) - 1)]
        gradients_biases = [np.sum(deltas[i], axis=0) for i in range(len(self.layers) - 1)]
        print("gradient_weights: ",gradients_weights)
        print("gradients_biases: ",gradients_biases)
        return gradients_weights, gradients_biases
    
    def update_weights(self, gradients_weights, gradients_biases, learning_rate):
        self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_weights)]
        self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_biases)]
    
    def train(self, X, Y, learning_rate, lam, max_iterations, epsilon):
        for iteration in range(max_iterations):
            activations = self.forward_pass(X)
            deltas = self.backward_pass(X, Y, activations)
            gradients_weights, gradients_biases = self.compute_gradients(activations, deltas)
            self.update_weights(gradients_weights, gradients_biases, learning_rate)
            # Compute cost function
            J = np.mean(np.square(activations[-1] - Y))
            print(f"Iteration {iteration+1}, Cost: {J}")
            # Check for convergence
            if J < epsilon:
                print("Converged!")
                break

    def accuracy(self, y_true, y_pred):
        correct = np.sum(y_true == y_pred)
        return correct / len(y_true)


    def f1_score(self, y_true, y_pred):
        tp = np.sum(np.logical_and(y_true, y_pred))
        fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))
        fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        return f1


    def evaluate(self, X_test, y_test):
        activations = self.forward_pass(X_test)[-1]
        y_pred = (activations > 0.5).astype(int)
        acc = self.accuracy(y_test, y_pred)
        f1 = self.f1_score(y_test, y_pred)
        return acc, f1


In [26]:
# Example usage
# Create a neural network with 2 input neurons, 3 hidden neurons, and 1 output neuron
nn = NeuralNetwork([2, 3, 1])

# Generate some dummy data for training
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y_train = np.array([[0], [1], [1], [0]])

# Train the neural network
nn.train(X_train, Y_train, learning_rate=0.1, lam=0.0, max_iterations=1000, epsilon=0.005)

# Generate some dummy data for testing
X_test = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y_test = np.array([[0], [1], [1], [0]])

# Evaluate the trained model
accuracy, f1_score = nn.evaluate(X_test, Y_test)
print("Accuracy:", accuracy)
print("F1 Score:", f1_score)

gradient_weights:  [array([[ 0.000402  , -0.00265863,  0.01294115],
       [-0.00080759, -0.00144509,  0.00444915]]), array([[0.00093006],
       [0.00027098],
       [0.00792345]])]
gradients_biases:  [array([ 0.00020471, -0.00166134,  0.00378406]), array([-0.00391733])]
Iteration 1, Cost: 0.24675711498480743
gradient_weights:  [array([[ 0.00040247, -0.00265482,  0.01295189],
       [-0.00080593, -0.00144155,  0.0044568 ]]), array([[0.00097211],
       [0.00028531],
       [0.00796393]])]
gradients_biases:  [array([ 0.00020595, -0.00165727,  0.00377606]), array([-0.00385331])]
Iteration 2, Cost: 0.24674243792757755
gradient_weights:  [array([[ 0.0004029 , -0.00265107,  0.01296282],
       [-0.00080426, -0.00143808,  0.00446456]]), array([[0.00101252],
       [0.00029873],
       [0.00800364]])]
gradients_biases:  [array([ 0.00020712, -0.00165336,  0.00376837]), array([-0.00379192])]
Iteration 3, Cost: 0.24672773681870974
gradient_weights:  [array([[ 0.00040329, -0.00264738,  0.0129739

Iteration 331, Cost: 0.23915545557488976
gradient_weights:  [array([[ 0.00019796, -0.00208519,  0.01748421],
       [-0.00025781, -0.00080788,  0.00844951]]), array([[ 0.00284996],
       [-0.00210963],
       [ 0.01600507]])]
gradients_biases:  [array([ 5.91451256e-05, -1.73168073e-03,  1.50250954e-03]), array([-0.00280039])]
Iteration 332, Cost: 0.23912230436031356
gradient_weights:  [array([[ 0.00019695, -0.00208403,  0.01749522],
       [-0.00025607, -0.00080608,  0.0084631 ]]), array([[ 0.00285407],
       [-0.00211953],
       [ 0.016027  ]])]
gradients_biases:  [array([ 5.86719959e-05, -1.73305096e-03,  1.48844618e-03]), array([-0.0028025])]
Iteration 333, Cost: 0.23908908575606508
gradient_weights:  [array([[ 0.00019593, -0.00208287,  0.01750621],
       [-0.00025434, -0.00080428,  0.00847668]]), array([[ 0.0028582 ],
       [-0.00212945],
       [ 0.01604891]])]
gradients_biases:  [array([ 5.81997481e-05, -1.73442724e-03,  1.47433888e-03]), array([-0.00280462])]
Iteration 334,

gradient_weights:  [array([[-3.08454811e-04, -1.62921238e-03,  1.95758953e-02],
       [ 2.11009104e-04, -3.63556972e-05,  1.22338294e-02]]), array([[ 0.00442323],
       [-0.00544206],
       [ 0.02103924]])]
gradients_biases:  [array([-2.24713695e-05, -2.43493255e-03, -3.66346402e-03]), array([-0.00360608])]
Iteration 626, Cost: 0.22642252392326906
gradient_weights:  [array([[-3.11178412e-04, -1.62675345e-03,  1.95793749e-02],
       [ 2.12259551e-04, -3.23583230e-05,  1.22451691e-02]]), array([[ 0.00443001],
       [-0.00545489],
       [ 0.02105038]])]
gradients_biases:  [array([-2.24703268e-05, -2.43849589e-03, -3.68095195e-03]), array([-0.00360911])]
Iteration 627, Cost: 0.22636951524517507
gradient_weights:  [array([[-3.13911064e-04, -1.62428452e-03,  1.95828320e-02],
       [ 2.13505955e-04, -2.83476278e-05,  1.22564930e-02]]), array([[ 0.0044368 ],
       [-0.00546773],
       [ 0.02106147]])]
gradients_biases:  [array([-2.24667578e-05, -2.44206813e-03, -3.69841336e-03]), arra

gradients_biases:  [array([ 4.04852769e-05, -3.36485478e-03, -6.42390116e-03]), array([-0.00410135])]
Iteration 831, Cost: 0.21440241716102842
gradient_weights:  [array([[-0.00109776, -0.00086231,  0.01981904],
       [ 0.00035504,  0.00112172,  0.01415096]]), array([[ 0.00608127],
       [-0.0082459 ],
       [ 0.02231807]])]
gradients_biases:  [array([ 4.11506054e-05, -3.37044404e-03, -6.43240094e-03]), array([-0.00410276])]
Iteration 832, Cost: 0.2143389935788365
gradient_weights:  [array([[-0.00110288, -0.00085713,  0.01981787],
       [ 0.00035505,  0.00112919,  0.01415782]]), array([[ 0.00609063],
       [-0.00826009],
       [ 0.0223193 ]])]
gradients_biases:  [array([ 4.18198942e-05, -3.37604433e-03, -6.44084848e-03]), array([-0.00410416])]
Iteration 833, Cost: 0.21427553347394945
gradient_weights:  [array([[-0.00110802, -0.00085193,  0.01981669],
       [ 0.00035506,  0.00113669,  0.01416464]]), array([[ 0.00610001],
       [-0.00827428],
       [ 0.02232048]])]
gradients_bias

In [28]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_pass(self, X):
        activations = [X]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(activations[-1], w) + b
            activations.append(self.sigmoid(z))
        return activations
    
    def backward_pass(self, X, Y, activations):
        deltas = [(activations[-1] - Y) * self.sigmoid_derivative(activations[-1])]
        for i in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(activations[i])
            deltas.insert(0, delta)
        return deltas
    
    def compute_gradients(self, activations, deltas):
        gradients_weights = [np.dot(activations[i].T, deltas[i]) for i in range(len(self.layers) - 1)]
        gradients_biases = [np.sum(deltas[i], axis=0) for i in range(len(self.layers) - 1)]
        return gradients_weights, gradients_biases
    
    def update_weights(self, gradients_weights, gradients_biases, learning_rate):
        self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_weights)]
        self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_biases)]
    
    def train(self, X, Y, learning_rate, lam, max_iterations, epsilon):
        for iteration in range(max_iterations):
            activations = self.forward_pass(X)
            deltas = self.backward_pass(X, Y, activations)
            gradients_weights, gradients_biases = self.compute_gradients(activations, deltas)
            self.update_weights(gradients_weights, gradients_biases, learning_rate)
            # Compute cost function
            J = np.mean(np.square(activations[-1] - Y))
            print(f"Iteration {iteration+1}, Cost: {J}")
            # Check for convergence
            if J < epsilon:
                print("Converged!")
                break

    def accuracy(self, y_true, y_pred):
        correct = np.sum(y_true == y_pred)
        return correct / len(y_true)


    def f1_score(self, y_true, y_pred):
        tp = np.sum(np.logical_and(y_true, y_pred))
        fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))
        fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        return f1


    def evaluate(self, X_test, y_test):
        activations = self.forward_pass(X_test)[-1]
        y_pred = (activations > 0.5).astype(int)
        acc = self.accuracy(y_test, y_pred)
        f1 = self.f1_score(y_test, y_pred)
        return acc, f1


In [30]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Load dataset
df_wine = pd.read_csv("/Users/noshitha/Downloads/hw3/datasets/hw3_wine.csv", delimiter="\t")

# Preprocess data
X_wine = pd.get_dummies(df_wine.iloc[:, 1:])
y_wine = df_wine.iloc[:, 0]

# Normalize data
y_wine_resized = y_wine.values.reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the target variable
y_encoded = encoder.fit_transform(y_wine_resized)

# Define model architectures
architectures = [
    [X_wine.shape[1], 5, 3, 3],  # Example architecture
]

# Instantiate the NeuralNetwork class with the desired architecture
model = NeuralNetwork(architectures[0])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_wine, y_encoded, test_size=0.2, random_state=42)

# Train the neural network
model.train(X_train, y_train, learning_rate=0.1, lam=0.0, max_iterations=1000, epsilon=0.005)

# Evaluate the trained model
accuracy, f1_score = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)
print("F1 Score:", f1_score)

Iteration 1, Cost: 0.3185688235920991
Iteration 2, Cost: 0.22582855598996
Iteration 3, Cost: 0.22141704327119757
Iteration 4, Cost: 0.22341253665759406
Iteration 5, Cost: 0.22515312045516978
Iteration 6, Cost: 0.23128550793769212
Iteration 7, Cost: 0.22664700055988482
Iteration 8, Cost: 0.23044094099948711
Iteration 9, Cost: 0.2247279060673223
Iteration 10, Cost: 0.22650944453852895
Iteration 11, Cost: 0.22370770213100724
Iteration 12, Cost: 0.22487805716848464
Iteration 13, Cost: 0.22287946793051072
Iteration 14, Cost: 0.22353905222232942
Iteration 15, Cost: 0.22216313301382512
Iteration 16, Cost: 0.22251544213334615
Iteration 17, Cost: 0.22157843734967922
Iteration 18, Cost: 0.22175028493534435
Iteration 19, Cost: 0.2211149931560793
Iteration 20, Cost: 0.22118518368490442
Iteration 21, Cost: 0.22075533827735155
Iteration 22, Cost: 0.22077155233953522
Iteration 23, Cost: 0.2204808226057023
Iteration 24, Cost: 0.22047086325888682
Iteration 25, Cost: 0.22027401782661266
Iteration 26, Co

Iteration 249, Cost: 0.21969847252529268
Iteration 250, Cost: 0.21969847252529268
Iteration 251, Cost: 0.21969847252529268
Iteration 252, Cost: 0.2196984725252927
Iteration 253, Cost: 0.21969847252529268
Iteration 254, Cost: 0.21969847252529268
Iteration 255, Cost: 0.21969847252529268
Iteration 256, Cost: 0.21969847252529268
Iteration 257, Cost: 0.21969847252529262
Iteration 258, Cost: 0.21969847252529268
Iteration 259, Cost: 0.21969847252529268
Iteration 260, Cost: 0.21969847252529268
Iteration 261, Cost: 0.21969847252529268
Iteration 262, Cost: 0.21969847252529268
Iteration 263, Cost: 0.2196984725252927
Iteration 264, Cost: 0.21969847252529268
Iteration 265, Cost: 0.21969847252529268
Iteration 266, Cost: 0.21969847252529262
Iteration 267, Cost: 0.21969847252529268
Iteration 268, Cost: 0.21969847252529268
Iteration 269, Cost: 0.21969847252529268
Iteration 270, Cost: 0.21969847252529268
Iteration 271, Cost: 0.21969847252529268
Iteration 272, Cost: 0.21969847252529268
Iteration 273, Cos

Iteration 508, Cost: 0.2196984725252927
Iteration 509, Cost: 0.2196984725252927
Iteration 510, Cost: 0.21969847252529268
Iteration 511, Cost: 0.2196984725252927
Iteration 512, Cost: 0.2196984725252927
Iteration 513, Cost: 0.2196984725252927
Iteration 514, Cost: 0.21969847252529268
Iteration 515, Cost: 0.2196984725252927
Iteration 516, Cost: 0.2196984725252927
Iteration 517, Cost: 0.2196984725252927
Iteration 518, Cost: 0.21969847252529268
Iteration 519, Cost: 0.2196984725252927
Iteration 520, Cost: 0.21969847252529268
Iteration 521, Cost: 0.2196984725252927
Iteration 522, Cost: 0.21969847252529268
Iteration 523, Cost: 0.2196984725252927
Iteration 524, Cost: 0.2196984725252927
Iteration 525, Cost: 0.21969847252529268
Iteration 526, Cost: 0.2196984725252927
Iteration 527, Cost: 0.2196984725252927
Iteration 528, Cost: 0.21969847252529268
Iteration 529, Cost: 0.2196984725252927
Iteration 530, Cost: 0.21969847252529268
Iteration 531, Cost: 0.21969847252529268
Iteration 532, Cost: 0.21969847

Iteration 754, Cost: 0.21969847252529268
Iteration 755, Cost: 0.21969847252529268
Iteration 756, Cost: 0.21969847252529268
Iteration 757, Cost: 0.21969847252529268
Iteration 758, Cost: 0.2196984725252927
Iteration 759, Cost: 0.2196984725252927
Iteration 760, Cost: 0.21969847252529268
Iteration 761, Cost: 0.21969847252529268
Iteration 762, Cost: 0.2196984725252927
Iteration 763, Cost: 0.21969847252529268
Iteration 764, Cost: 0.21969847252529265
Iteration 765, Cost: 0.2196984725252927
Iteration 766, Cost: 0.21969847252529268
Iteration 767, Cost: 0.21969847252529268
Iteration 768, Cost: 0.2196984725252927
Iteration 769, Cost: 0.2196984725252927
Iteration 770, Cost: 0.21969847252529268
Iteration 771, Cost: 0.2196984725252927
Iteration 772, Cost: 0.2196984725252927
Iteration 773, Cost: 0.2196984725252927
Iteration 774, Cost: 0.21969847252529268
Iteration 775, Cost: 0.2196984725252927
Iteration 776, Cost: 0.21969847252529268
Iteration 777, Cost: 0.2196984725252927
Iteration 778, Cost: 0.2196

In [35]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = None
        self.biases = None
        self.initialize_network()

    def initialize_network(self):
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_pass(self, X):
        X = np.array(X)  # Ensure X is converted to numpy array
        activations = [X]
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            print(f"\nLayer {i+1}:")
            print("X:", X.dtype)
            print("Weights:", w.dtype)
            print("Biases:", b.dtype)
            z = np.dot(activations[-1], w) + b
            print("Z:", z.dtype)
            activations.append(self.sigmoid(z))
        return activations

    
    def backward_pass(self, X, Y, activations):
        deltas = [(activations[-1] - Y) * self.sigmoid_derivative(activations[-1])]
        for i in range(len(self.layers) - 2, 0, -1):
            delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(activations[i])
            deltas.insert(0, delta)
        return deltas
    
    def compute_gradients(self, activations, deltas):
        gradients_weights = [np.dot(activations[i].T, deltas[i]) for i in range(len(self.layers) - 1)]
        gradients_biases = [np.sum(deltas[i], axis=0) for i in range(len(self.layers) - 1)]
        return gradients_weights, gradients_biases
    
    def update_weights(self, gradients_weights, gradients_biases, learning_rate):
        self.weights = [w - learning_rate * gw for w, gw in zip(self.weights, gradients_weights)]
        self.biases = [b - learning_rate * gb for b, gb in zip(self.biases, gradients_biases)]
    
    def train(self, X, Y, learning_rate, lam, max_iterations, epsilon):
        for iteration in range(max_iterations):
            activations = self.forward_pass(X)
            deltas = self.backward_pass(X, Y, activations)
            gradients_weights, gradients_biases = self.compute_gradients(activations, deltas)
            self.update_weights(gradients_weights, gradients_biases, learning_rate)
            # Compute cost function
            J = np.mean(np.square(activations[-1] - Y))
            #print(f"Iteration {iteration+1}, Cost: {J}")
            # Check for convergence
            if J < epsilon:
                print(f"Converged at cost :{J} while Epsilon:{epsilon} ")
                return J
        return J
            

    def accuracy(self, y_true, y_pred):
        correct = np.sum(np.all(y_true == y_pred, axis=1))
        return correct / len(y_true)


    def f1_score(self, y_true, y_pred):
        tp = np.sum(np.logical_and(y_true, y_pred))
        fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))
        fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        return f1


    def evaluate(self, X_test, y_test, J):
        activations = self.forward_pass(X_test)[-1]
        y_pred = (activations > 0.5).astype(int)
        acc = self.accuracy(y_test, y_pred)
        f1 = self.f1_score(y_test, y_pred)
        return J, acc, f1
    
    def k_fold_cross_validation(self, X, y, architectures, regularization_params, learning_rate, max_iterations, epsilon):
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        results_accuracy = {}
        results_f1_score = {}

        for arch in architectures:
            for lam in regularization_params:
                accuracy_list = []
                f1_score_list = []
                for train_index, test_index in skf.split(X, y):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                    mean = np.mean(X_train, axis=0)
                    std = np.std(X_train, axis=0)
                    X_train_normalized = (X_train - mean) / std
                    X_test_normalized = (X_test - mean) / std

                    model = NeuralNetwork(arch)
                    model.train(X_train_normalized, y_train, learning_rate=learning_rate, lam=lam, max_iterations=max_iterations, epsilon=epsilon)
                    accuracy, f1_score = model.evaluate(X_test_normalized, y_test)
                    accuracy_list.append(accuracy)
                    f1_score_list.append(f1_score)

                mean_accuracy = np.mean(accuracy_list)
                mean_f1_score = np.mean(f1_score_list)

                results_accuracy[(str(arch), lam)] = mean_accuracy
                results_f1_score[(str(arch), lam)] = mean_f1_score

        return results_accuracy, results_f1_score

# Define the network architectures and regularization parameters
architectures = [[1, 2, 1], [2, 4, 3, 2]]
regularization_params = [0.000, 0.250]

# Create an instance of the NeuralNetwork class
model = NeuralNetwork([])

# Output the results
for i, arch in enumerate(architectures):
    for lam in regularization_params:
        print(f"backprop_example_{i+1}: Regularization parameter lambda={lam}\n")

        # Initialize the network with the given architecture
        model.weights = [np.random.randn(arch[i], arch[i+1]) for i in range(len(arch)-1)]
        model.biases = [np.zeros((1, arch[i+1])) for i in range(len(arch)-1)]

        # Output initial weights
        print("Initializing the network with the following structure (number of neurons per layer):", arch)
        for j in range(len(arch) - 1):
            print(f"\nInitial Theta{j+1} (the weights of each neuron, including the bias weight, are stored in the rows):")
            for k in range(len(model.weights[j])):
                print("\t", end="")
                print(*model.weights[j][k], sep="  ")

        # Training set
        print("\nTraining set")
        for idx, instance in enumerate(X_train):
            print(f"\tTraining instance {idx+1}")
            print("\t\tx:", instance)
            print("\t\ty:", y_train[idx])

        # Compute and output the error/cost J
        print("\n--------------------------------------------")
        print("Computing the error/cost, J, of the network")
        for idx, instance in enumerate(X_train):
            print(f"\tProcessing training instance {idx+1}")
            activations = model.forward_pass(instance)
            print("\tForward propagating the input", instance)
            for l, activation in enumerate(activations[1:], start=1):
                print(f"\t\ta{l}:", activation)
            print(f"\n\tf(x): {activations[-1][0]}")
            print(f"Predicted output for instance {idx+1}:", activations[-1][0])
            print(f"Expected output for instance {idx+1}:", y_train[idx][0])
            print(f"Cost, J, associated with instance {idx+1}: {np.mean(np.square(activations[-1] - y_train[idx])):.3f}")

        # Final (regularized) cost J
        print(f"\nFinal (regularized) cost, J, based on the complete training set: {J:.5f}")

        # Running backpropagation
        print("\n--------------------------------------------")
        print("Running backpropagation")
        for idx, instance in enumerate(X_train):
            print(f"\tComputing gradients based on training instance {idx+1}")
            activations = model.forward_pass(instance)
            deltas = model.backward_pass(instance, y_train[idx], activations)
            for l, delta in enumerate(deltas[1:], start=2):
                print(f"\t\tdelta{l}:", delta[0])
            for l, gradient_weights in enumerate(model.compute_gradients(activations, deltas)[0], start=1):
                print(f"\n\t\tGradients of Theta{l} based on training instance {idx+1}:")
                for k in range(len(gradient_weights)):
                    print("\t\t", end="")
                    print(*gradient_weights[k], sep="  ")
        print("\n--------------------------------------------")


NameError: name 'layers' is not defined